In [236]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import dateutil.parser
import json
from urllib2 import Request, urlopen
from pandas.io.json import json_normalize
from pprint import pprint
%matplotlib inline

In [5]:
divvy_df = pd.read_csv('Divvy_Trips_2014-Q3-07.csv', sep = ',')

In [209]:
divvy_df.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,2886259,7/31/2014 23:56,8/1/2014 0:03,2602,386,291,Wells St & Evergreen Ave,53,Wells St & Erie St,Subscriber,Female,1979
1,2886258,7/31/2014 23:58,8/1/2014 0:07,2403,495,98,LaSalle St & Washington St,106,State St & Pearson St,Subscriber,Male,1974
2,2886257,7/31/2014 23:58,8/1/2014 2:10,669,7947,240,Sheridan Rd & Irving Park Rd,240,Sheridan Rd & Irving Park Rd,Customer,NaN,NaN
3,2886256,7/31/2014 23:58,8/1/2014 0:19,2431,1282,47,State St & Kinzie St,14,Morgan St & 18th St,Customer,NaN,NaN
4,2886255,7/31/2014 23:57,8/1/2014 2:10,2885,7972,240,Sheridan Rd & Irving Park Rd,240,Sheridan Rd & Irving Park Rd,Customer,NaN,NaN


In [327]:
divvy_df['from_station_id'].describe()

#there are 300 unique station IDs but the range is from 5 to 351

count    410340.000000
mean        161.966706
std          99.753870
min           5.000000
25%          75.000000
50%         154.000000
75%         249.000000
max         351.000000
Name: from_station_id, dtype: float64

In [21]:
divvy_hr_df = divvy_df.copy(deep = True)

In [28]:
def hourbinner(fulltime):
    return dateutil.parser.parse(fulltime).replace(minute=0, second=0)
    
# print dateutil.parser.parse('7/31/2014 23:58').replace(minute=0, second=0)

In [29]:
divvy_hr_df.starttime = divvy_hr_df.starttime.apply(lambda x: hourbinner(x))

In [32]:
divvy_hr_df.stoptime = divvy_hr_df.stoptime.apply(lambda x: hourbinner(x))

In [49]:
print divvy_hr_df.groupby(['from_station_name'])['starttime'].size().order(ascending = False)

from_station_name
Streeter Dr & Illinois St       11149
Lake Shore Dr & Monroe St        8533
Theater on the Lake              8358
Michigan Ave & Oak St            7338
Lake Shore Dr & North Blvd       6872
Millennium Park                  5560
Clinton St & Washington Blvd     5237
Canal St & Adams St              4840
Columbus Dr & Randolph St        4672
Museum Campus                    4670
Canal St & Madison St            4336
McClurg Ct & Illinois St         3889
Michigan Ave & Washington St     3794
Clark St & Elm St                3760
Indiana Ave & Roosevelt Rd       3545
Wells St & Concord Ln            3438
Michigan Ave & Jackson Blvd      3339
Daley Center Plaza               3228
Franklin St & Arcade Pl          3211
State St & Erie St               3208
Lake Shore Dr & Ohio St          3179
Dearborn St & Adams St           3177
Michigan Ave & Lake St           3113
Clark St & Lincoln Ave           3090
Michigan Ave & Pearson St        3066
State St & Kinzie St            

In [146]:
fromstation_starttime = divvy_hr_df.groupby(['from_station_name', 'starttime']).size()

In [147]:
ts = fromstation_starttime.unstack(level=0)

In [149]:
index = pd.date_range('2014-07-01', '2014-07-31', freq='H')

In [150]:
ts = ts.reindex(index, fill_value = 0)

In [154]:
ts = ts.fillna(0)

In [156]:
ts = ts.stack()

In [157]:
ts = ts.swaplevel(0,1)

In [158]:
ts = ts.sortlevel()

In [165]:
tostation_stoptime = divvy_hr_df.groupby(['to_station_name', 'stoptime']).size()

In [166]:
vs = tostation_stoptime.unstack(level=0)

In [167]:
vs = vs.reindex(index, fill_value = 0)

In [168]:
vs = vs.fillna(0)

In [169]:
vs = vs.stack()

In [170]:
vs = vs.swaplevel(0,1)

In [171]:
vs = vs.sortlevel()

In [200]:
combined = pd.concat([ts, vs], axis=1)

In [201]:
combined.columns = ['outflow', 'inflow']

In [202]:
combined.head()

outflow  inflow
from_station_name                                     
900 W Harrison    2014-07-01 00:00:00        0       0
                  2014-07-01 01:00:00        0       0
                  2014-07-01 02:00:00        0       0
                  2014-07-01 03:00:00        0       0
                  2014-07-01 04:00:00        0       0

In [205]:
combined['flowdiff'] = combined['inflow'] - combined['outflow']

In [252]:
combined.index.names = ['station_name', 'hourly_time']

In [254]:
combined.head()

outflow  inflow  flowdiff
station_name   hourly_time                                   
900 W Harrison 2014-07-01 00:00:00        0       0         0
               2014-07-01 01:00:00        0       0         0
               2014-07-01 02:00:00        0       0         0
               2014-07-01 03:00:00        0       0         0
               2014-07-01 04:00:00        0       0         0

In [214]:
combined['flowdiff'].value_counts()

 0     101030
-1      27066
 1      26123
-2      13577
 2      13067
-3       6909
 3       6642
 4       3949
-4       3853
-5       2323
 5       2244
 6       1436
-6       1342
-7        901
 7        852
-8        592
 8        542
-9        422
 9        379
-10       288
 10       271
 11       247
-11       207
-12       175
 12       154
 13       148
-13       124
 15       110
 16        96
 14        93
        ...  
-37         5
-43         5
-34         5
 32         4
-40         3
-42         3
 46         3
-49         3
 52         3
-45         2
 47         2
 45         2
 42         2
 64         1
 41         1
 49         1
 51         1
 53         1
 57         1
 59         1
 71         1
-59         1
-57         1
-56         1
-55         1
-50         1
-48         1
-47         1
-46         1
 68         1
dtype: int64

In [272]:
#flattened the multi-index DF from above.

combined.reset_index(inplace=True)  

In [283]:
combined.shape

(216300, 5)

In [328]:
combined.head()

,station_name,hourly_time,outflow,inflow,flowdiff
0,900 W Harrison,2014-07-01 00:00:00,0,0,0
1,900 W Harrison,2014-07-01 01:00:00,0,0,0
2,900 W Harrison,2014-07-01 02:00:00,0,0,0
3,900 W Harrison,2014-07-01 03:00:00,0,0,0
4,900 W Harrison,2014-07-01 04:00:00,0,0,0


In [292]:
just_ID_name = divvy_df.copy(deep=True)[['from_station_id', 'from_station_name']]

In [293]:
just_ID_name = just_ID_name.rename(columns = {'from_station_name':'station_name'})

In [321]:
just_ID_name = just_ID_name.drop_duplicates()

In [329]:
just_ID_name.head()

,from_station_id,station_name
0,291,Wells St & Evergreen Ave
1,98,LaSalle St & Washington St
2,240,Sheridan Rd & Irving Park Rd
3,47,State St & Kinzie St
5,28,Larrabee St & Menomonee St


In [322]:
bigcombine = combined.copy(deep=True)

In [323]:
result = bigcombine.merge(just_ID_name)

In [332]:
result = result.rename(columns = {'from_station_id':'id'})

In [333]:
result.head()

,station_name,hourly_time,outflow,inflow,flowdiff,id
0,900 W Harrison,2014-07-01 00:00:00,0,0,0,109
1,900 W Harrison,2014-07-01 01:00:00,0,0,0,109
2,900 W Harrison,2014-07-01 02:00:00,0,0,0,109
3,900 W Harrison,2014-07-01 03:00:00,0,0,0,109
4,900 W Harrison,2014-07-01 04:00:00,0,0,0,109


In [342]:
type(result.columns.values[0])

str

In [359]:
result.columns.values

array(['station_name', 'hourly_time', 'outflow', 'inflow', 'flowdiff', 'id'], dtype=object)

In [242]:
request= Request('http://www.divvybikes.com/stations/json')
response = urlopen(request)
api = response.read()
api_data = json.loads(api)
api_df = json_normalize(api_data['stationBeanList'])

In [243]:
api_df = api_df[["id", "latitude", "longitude", "stationName", "totalDocks"]]

In [268]:
api_df.head()

,id,latitude,longitude,stationName,totalDocks
0,2,41.872293,-87.624091,Michigan Ave & Balbo Ave,35
1,3,41.867226,-87.615355,Shedd Aquarium,31
2,4,41.856268,-87.613348,Burnham Harbor,22
3,5,41.874053,-87.627716,State St & Harrison St,23
4,6,41.885042,-87.612795,Dusable Harbor,31


In [370]:
api_df.columns

Index([u'id', u'latitude', u'longitude', u'stationName', u'totalDocks'], dtype='object')

In [371]:
result.columns

Index([u'station_name', u'hourly_time', u'outflow', u'inflow', u'flowdiff',
       u'id'],
      dtype='object')

In [368]:
api_df.columns = map(lambda x: x.encode('utf-8'), api_df.columns)

In [374]:
bigresult = pd.merge(result, api_df, on='id')

In [480]:
# bigresult.drop(bigresult.columns[[-1]], axis = 1, inplace = True)
# bigresult.head()

,station_name,hourly_time,outflow,inflow,flowdiff,id,latitude,longitude,totalDocks
0,900 W Harrison,2014-07-01 00:00:00,0,0,0,109,41.874675,-87.650019,19
1,900 W Harrison,2014-07-01 01:00:00,0,0,0,109,41.874675,-87.650019,19
2,900 W Harrison,2014-07-01 02:00:00,0,0,0,109,41.874675,-87.650019,19
3,900 W Harrison,2014-07-01 03:00:00,0,0,0,109,41.874675,-87.650019,19
4,900 W Harrison,2014-07-01 04:00:00,0,0,0,109,41.874675,-87.650019,19


In [486]:
bigresult['running_low'] = ((bigresult.flowdiff < 0) & (bigresult.flowdiff.abs() > (bigresult.totalDocks * 1.5/3.0))).astype(int)

In [487]:
bigresult.head()

,station_name,hourly_time,outflow,inflow,flowdiff,id,latitude,longitude,totalDocks,running_low
0,900 W Harrison,2014-07-01 00:00:00,0,0,0,109,41.874675,-87.650019,19,0
1,900 W Harrison,2014-07-01 01:00:00,0,0,0,109,41.874675,-87.650019,19,0
2,900 W Harrison,2014-07-01 02:00:00,0,0,0,109,41.874675,-87.650019,19,0
3,900 W Harrison,2014-07-01 03:00:00,0,0,0,109,41.874675,-87.650019,19,0
4,900 W Harrison,2014-07-01 04:00:00,0,0,0,109,41.874675,-87.650019,19,0


In [490]:
bigresult['running_low'].describe()

count    216300.000000
mean          0.005039
std           0.070809
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: running_low, dtype: float64